# Input Helper AutoGenerate Configurations from CSV

This notebook will focus on the Input_helper configuration generation for us with the plant integration within Home Assistant.

Date of Project: July 24 2022
Software and Devices Used: 
- Home Assistant 2022.7
- Awair Element AQI Sensor
- Hue Motion Sensor + Hue Bridge

Further notebooks will be created to document the other portions of this project which will be available on my Github repositoriy located here.

https://github.com/netmanchris/HassPlants



# Miflora BLE Plant Sensor + ESP32 Config Generation for Home Assistant

Like most things, what was easy at low volume can start to show challenges when reaching certain scales. 

This project is intended to help automate the creation of my 
home assistant based Plant monitoring setup. Although I prefer to use the Xioami Miflora sensors who's initial setup can be seen documented in this [project](./README.MD).

This has been a work in progress and will likely change as I experiment. 

## WHY?

### Why part 1?

I killed plants. I'd like my plants to not die.

### Why part 2?

Why not? Home Automation is supposed to help us and make our lives easier. 

### Why part 3?

MifLora Sensors are kinda awesome, but at scale they can start to be very expensive. When I started this project last year, they could be found as low as $11 dollars a piece. At the time of this writting, they are showing on AliExpress at $26 average and somewhere around $50 on Amazon Canada.  Too rich for my blood.


### Simple Solution

For users not wanting to invest in the ESP32 + Miflora sensors. You can leverage a combination of a traditional plant sensor to take and track manual readings in combination of existing temperature and light sensors that you likely might already have in your homes.




### CSV File Formating and Input Generation

There are currently two files which are required as data inputs for this project

- plants_worksheet.csv is the file where the plants to include in the generation of the input_helpers will be stored




# Plants Worksheet CSV

The Plants worksheet is a CSV which contains all of the information required to generate the ESP32 configurations, plant integration configurations, as well as the dashboards for both desktop/tablet and mobile formats.

The plant worksheet can be located here.

Each row of this worksheet represents a single plant with the following attributes\

- name: Unique Value. One per plant. type: String
- plant_type: One per plant. Must match values in the *plant_attributes.csv* worksheet type: String
- room: Grouping mechanism. Plants with the same value will be automatically grouped together.
- sensor: Unique Value. One per sensor. type: Integer
- sensor_number: Unique Value. One per sensor. This will be appended in the ESP config as the unique portion of the auto-generated sensor entity.  type: String
- sensor_mac: the 12 digit mac-address in the format XX:XX:XX:XX:XX:XX example C4:7C:8D:6D:8D:F6 type: String
- esp_parent: {FUTURE USE} Intention is to use this to create a permanent binding between a specific plant sensor and a specific ESP32 device to prevent naming conflicts when dealing with larger rooms ( > 13 plants per room ) type: String 
- esp_current: Unused. Documentation purposes only. type: String
- room_picture: {FUTURE USE} Intention is to use this to bind specific plants to specific room pictures for building - out dashboards that require multiple camera angles. type: String
- top: Numerical value used a the X coodinate for plant placement for the room picture generation. type: Integer
- left: Numerical value used a the Y coodinate for plant placement for the room picture generation. type: Integer
- flash_light: full name of the script entitiy used to flash the lights where a specific plant is located. type: String







## Creating the Input Helpers

In this step we'll use a jinja2 template to create the input_helpers.

Need to create an input_helper for each non-sensor plant.

For organization, I'm thinking of creating a folder called "plants" in a folder called "helpers" which will existin the "entities" folder fo my root configuration

```
config
   ->entities
     -> helpers
       -> plants
```
       
This should also require adding a line to the configuration.yaml file to make sure that the structure is included in the processing of the overall configuration.


```input_number: !include_dir_merge_named entities/helpers```



## Proper Format

The helpers need to be in valid YAML format. I created examples manually to use as the pattern for the jinja2 template

ho_snake1_conductivity:
    name: ho_snake1_conductivity
    min: 0
    max: 5000
    step: 1
    mode: box
    unit_of_measurement: µS/cm

ho_snake1_humidity:
    name: ho_snake1_humidity
    min: 0
    max: 100
    step: 1
    mode: box
    unit_of_measurement: '%'
    
    

## Tested Template

The Plants worksheets includes an attribute called *sensor_model*  If this attribute is defined as *manual* in the worksheet file, then the following template will render a single file for each of room defined in the spreadsheet. 

```
{% for plant in plants %}{% if plant.sensor_model == 'manual' %}
{{plant.name}}_conductivity:
    name: {{plant.name}}_conductivity
    min: 0
    max: 5000
    step: 1
    mode: box
    unit_of_measurement: µS/cm

{{plant.name}}_humidity:
    name: {{plant.name}}_humidity
    min: 0
    max: 100
    step: 1
    mode: box
    unit_of_measurement: '%'

{% endif %} {% endfor %}




# Generating the Configuration Files

In this section, we'll go through the python code used to generate the actual configurations



In [15]:
import pandas as pd
from jinja2 import Template
import os

# Load Plants CSV

In this section, we will load the plants CSV file which will be used to create the ESPHome configurations to bind the Miflora BLE Plant sensors to a specific ESP32. 

There is one file needed here.

1. plants_worksheet.csv which includes the various plants, sensors, room locations, corresponding script to flash lights.







In [16]:
#plants_df = pd.read_csv('data/plants_worksheet.csv', index_col=0)
plants_df = pd.read_csv('data/plants_worksheet_sample.csv', index_col=0)

## Create Rooms List

In this step, we will create a list of the room names defined in the CSV file. Defining the room names logically is important in that ESP32 configurations, Plant entity definitions, and dashboards for each individual room will be automatically generated through this code. Spending some time thinking about how you want to organize your plants dashboards is a very, very good idea.

Note: I've got logic in the jinja2 files that will check to see whether the element in the room list is of type "string" which will allow sensors unassigned to rooms to be ignored. 

TODO - CHECK TO MAKE SURE THIS IS TRUE FOR ALL TEMPLATES

In [17]:
rooms_list = list(plants_df.room.unique())
rooms_list

['boys_bedroom',
 'dining_room',
 'downstairs_bathroom',
 'games_room',
 'home_office',
 'kitchen',
 'living_room',
 'master_bedroom',
 nan]

## Loading the Input Helper Jinja2 Templates

In this step, we load the input_helper specific templates from the templates folder and load it as a jinja2 template.


In [18]:
with open('templates/input_helpers.j2') as f:
    input_helper_template = f.read()
j2_input_helper_template = Template(input_helper_template)

## Rendering Input Helper Configurations

We will now render the Input Helper Configurations.

Note: As a prerequisite, the directory structure has to preexist to be able to properly save the file configurations to disk. 

Make sure that the folder "input_helpers" exists under the output folder.





In [19]:
for room in rooms_list:
    if type(room) is str:
        room = [room]
        room_df = plants_df[plants_df.room.isin(room)]
        #print (len(room_df))
        if len(room_df) > 0:
            #print (room)
            room_dict = room_df.to_dict('records')
            #print (len(room_dict))

            # OUTPUT THE ESP32 ESPHome Configuration
            with open('output/input_helpers/'+room_dict[0]['room']+'.txt', 'w') as f:
                f.write(j2_input_helper_template.render(plants=room_dict))

# Sample Output

The code above should generate a single file per room which contains all of the required input_helpers.

The following is the a sample of the file labeled "boys_bedroom" .txt.

```
bbd_snake_1_soil_conductivity:
    name: bbd_snake_1_soil_conductivity
    min: 0
    max: 5000
    step: 1
    mode: box
    unit_of_measurement: µS/cm

bbd_snake_1_moisture:
    name: bbd_snake_1_moisture
    min: 0
    max: 100
    step: 1
    mode: box
    unit_of_measurement: '%'
```

The configuration should be cut and paste into a file labeled %room_name%.yaml in the appropriate folder

```
config
   ->entities
     -> helpers
       -> plants
```
       
This should also require adding a line to the configuration.yaml file to make sure that the structure is included in the processing of the overall configuration.


```input_number: !include_dir_merge_named entities/helpers```

![image](./images/studiocode_inputhelpers.png)


Once you reload your configuration or restart your home assistant instance, you should be able to verify that the input_helpers are shown as follows


Navigate to **Settings > Devices & Services > Helpers** to validate that the input helpers were created as desired.

![image](./images/input_helper_entities.png)

# Things to Note

The Input_helpers will be created with default values of 0.  You may be tempted to use the "default" keyword in the yaml configuration, but that will reset the input_helper value to 0 every time you restart Home Assisant. If the goal here is to be able to track the status of specific plants over a period of time, we need those values to persist through reboots.


